In [3]:
import os
import shutil
import numpy as np
import torch.nn as nn
from torch.autograd import Variable

In [5]:
os.listdir(os.getcwd())

['ros',
 'models',
 '__init__.py',
 'Untitled.ipynb',
 '.ipynb_checkpoints',
 'main.py',
 'utils',
 'data']

In [14]:
t = os.path.join(os.getcwd(), 'results')
print t
if os.path.isdir(t):
    print 'yes'
else:
    print 'no'

/home/lex/catkin_ws/src/soft-neuro-adapt/pyrnn/src/results
no


In [9]:
help(os.path.join)

Help on function join in module posixpath:

join(a, *p)
    Join two or more pathname components, inserting '/' as needed.
    If any component is an absolute path, all previous path components
    will be discarded.  An empty last part will result in a path that
    ends with a separator.



In [16]:
import csv
help(csv.writer)

Help on built-in function writer in module _csv:

writer(...)
    csv_writer = csv.writer(fileobj [, dialect='excel']
                                [optional keyword args])
        for row in sequence:
            csv_writer.writerow(row)
    
        [or]
    
        csv_writer = csv.writer(fileobj [, dialect='excel']
                                [optional keyword args])
        csv_writer.writerows(rows)
    
    The "fileobj" argument can be any object that supports the file API.

